<a href="https://colab.research.google.com/github/BlackMagicAI/Tiny-Yolo-3/blob/master/Tiny_Yolo3_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tiny yolo3 Colab Notebook

**Outline of Steps**

[keras-yolo2](https://github.com/experiencor/keras-yolo2)

[keras on google cloud ML](https://stackoverflow.com/questions/41959318/deploying-keras-models-via-google-cloud-ml)

[Yolo3 keras github](https://github.com/xiaochus/YOLOv3/blob/master/yad2k.py)

[A Practical Guide to Object Detection using the Popular YOLO Framework](https://www.analyticsvidhya.com/blog/2018/12/practical-guide-object-detection-yolo-framewor-python/)



##Reference

	@article{YOLOv3,  
	  title={YOLOv3: An Incremental Improvement},  
	  author={J Redmon, A Farhadi },
	  year={2018}
    url={https://pjreddie.com/media/files/papers/YOLOv3.pdf}


Redmon, Joseph. “Yolo Web: Real-Time Object Detection.” YOLO: Real-Time Object Detection, Joseph Redmon, 2018, pjreddie.com/darknet/yolo/.

Based on code from
Xiaochus, Larry. “YOLOv3” Github code, Xiaochus, Larry, 2018, https://github.com/xiaochus/YOLOv3.

(https://machinethink.net/blog/object-detection-with-yolo/)

    + Initialization
        + Download COCO detection data from http://cocodataset.org/#download
            + http://images.cocodataset.org/zips/train2014.zip <= train images
            + http://images.cocodataset.org/zips/val2014.zip <= validation images
            + http://images.cocodataset.org/annotations/annotations_trainval2014.zip <= train and validation annotations
        + Run this script to convert annotations in COCO format to VOC format
            + https://gist.github.com/chicham/6ed3842d0d2014987186#file-coco2pascal-py
        + Download pre-trained weights from https://pjreddie.com/darknet/yolo/
            + https://pjreddie.com/media/files/yolo.weights
        + Specify the directory of train annotations (train_annot_folder) and train images (train_image_folder)
        + Specify the directory of validation annotations (valid_annot_folder) and validation images (valid_image_folder)
        + Specity the path of pre-trained weights by setting variable *wt_path*
    + Construct equivalent network in Keras
        + Network arch from https://github.com/pjreddie/darknet/blob/master/cfg/yolo-voc.cfg
    + Load the pretrained weights
    + Perform training 
    + Perform detection on an image with newly trained weights
    + Perform detection on an video with newly trained weights

# IMPORTANT!!!
Read comment in next cell before procedding.

In [ ]:
#!!!!IMPORTANT - RUN THIS CELL FIRST AND RESTART THE RUNTIME BEOFORE RUNNING THE OTHER CELLS
!pip install tensorflow==1.15.3 #this works

In [ ]:
!git clone https://github.com/xiaochus/YOLOv3.git
  
##Make training data directories
!mkdir yolo3_tiny

%cd yolo3_tiny

# Make image input and output directories
!mkdir images
!mkdir out

!cp ../YOLOv3/images/test/*.jpg images

!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3-tiny.cfg
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

# copy classes file to yolo3_tiny directory
!cp ../YOLOv3/data/coco_classes.txt coco_classes.txt

#Get custom fonts for image annotations
!wget -O font.zip https://fonts.google.com/download?family=Ubuntu
!unzip font.zip -d fonts

# Initialization

##Imports

In [ ]:
#numpy is a math library use to create and manipulate matricies
import numpy as np
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, Reshape, LeakyReLU, MaxPooling2D, UpSampling2D, Lambda
from keras.models import Model
from keras.layers.merge import concatenate
from keras.regularizers import l2
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from PIL import Image, ImageDraw, ImageFont

# ML Cloud serving imports
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants, signature_def_utils_impl, utils
from functools import partial


Using TensorFlow backend.


In [ ]:
# print("Keras version " + keras.__version__)
print("Tensorflow version" + tf.__version__) #should be 1.15.3
!python --version #3.6.9 works

Tensorflow version1.15.3
Python 3.6.9


In [ ]:
LABELS = ['raccoon', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
# LABELS = ['person']

IMAGE_H, IMAGE_W = 416, 416
GRID_H,  GRID_W  = 13 , 13
BOX              = 5
CLASS            = len(LABELS)
CLASS_WEIGHTS    = np.ones(CLASS, dtype='float32')
OBJ_THRESHOLD    = 0.3#0.5
NMS_THRESHOLD    = 0.3#0.45
ANCHORS          = [10,14, 23,27, 37,58, 81,82, 135,169, 344,319]

NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
CLASS_SCALE      = 1.0

BATCH_SIZE       = 16
WARM_UP_BATCHES  = 0
TRUE_BOX_BUFFER  = 50

In [ ]:
# wt_path = 'yolov2.weights'  
# wt_path = 'yolov3.weights'
wt_path = 'yolov3-tiny.weights' 
train_image_folder = './data/coco/train/'
train_annot_folder = './data/coco/trainann/'
valid_image_folder = './data/coco/val/'
valid_annot_folder = './data/coco/valann/'

# Construct the network - Model

In [ ]:
# the function to implement the orgnization layer (thanks to github.com/allanzelener/YAD2K)
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [ ]:
# weights_file.close()
# weights function
def setWeights(filters ,size ,count , input_shape, weights_file, doBatch):
  # ********Start weights code*************
  prev_layer_shape = K.int_shape(input_shape)

  weights_shape = (size, size, prev_layer_shape[-1], filters)
  darknet_w_shape = (filters, weights_shape[2], size, size)
  weights_size = np.product(weights_shape)

  conv_bias = np.ndarray(
    shape=(filters, ),
    dtype='float32',
    buffer=weights_file.read(filters * 4))
  count += filters

  conv_weights = []
  bn_weight_list = []

# apply batch normalization of doBatch flag is true
  if doBatch:

    bn_weights = np.ndarray(
        shape=(3, filters),
        dtype='float32',
        buffer=weights_file.read(filters * 12))
    count += 3 * filters

    # TODO: Keras BatchNormalization mistakenly refers to var
    # as std.
    bn_weight_list = [
        bn_weights[0],  # scale gamma
        conv_bias,  # shift beta
        bn_weights[1],  # running mean
        bn_weights[2]  # running var
    ]

  conv_weights = np.ndarray(
      shape=darknet_w_shape,
      dtype='float32',
      buffer=weights_file.read(weights_size * 4))
  count += weights_size

  conv_weights = np.transpose(conv_weights, [2, 3, 1, 0])

  #conv_weights = []
  if doBatch:
    conv_weights = [conv_weights] #if BatchNormalization use this
  else:
    conv_weights = [conv_weights, conv_bias] #else use this

  return conv_weights, bn_weight_list, count

# ********End weights code*************

In [ ]:
# https://github.com/xiaochus/YOLOv3/blob/master/yad2k.py
weights_file = open(wt_path, 'rb')
weights_header = np.ndarray(
        shape=(5, ), dtype='int32', buffer=weights_file.read(20))
print('Weights Header: ', weights_header)

Weights Header:  [       0        2        0 32013312        0]


In [ ]:
sess = tf.Session()
from keras import backend as K
K.set_session(sess)
# K.set_learning_phase(0) #all new operations will be in test mode from now on - The learning phase flag is a bool tensor (0 = test, 1 = train) to be passed as input to any Keras function that uses a different behavior at train time and test time

input_image = Input(shape=(IMAGE_H, IMAGE_W, 3), name='input_1')
true_boxes  = Input(shape=(1, 1, 1, TRUE_BOX_BUFFER , 4))
weight_decay = 0.0005

#Initialize filter count variable
count = 0
# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(16, 3, count, input_image ,weights_file , True)


# Layer 1
x = Conv2D(16, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_1', use_bias=False)(input_image)
x = BatchNormalization(weights=bn_weight_list, name='norm_1')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(32 ,3 ,count ,x ,weights_file , True)

# Layer 2
x = Conv2D(32, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_2', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_2')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(64 ,3 ,count , x,weights_file , True)

# Layer 3
x = Conv2D(64, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_3', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_3')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(128 ,3 ,count , x,weights_file , True)

# Layer 4
x = Conv2D(128, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_4', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_4')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(256 ,3 ,count ,x ,weights_file , True)

# Layer 5
x = Conv2D(256, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_5', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_5')(x)
x = LeakyReLU(alpha=0.1)(x)

convd5 = x

x = MaxPooling2D(pool_size=(2, 2))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(512 ,3 ,count ,x ,weights_file , True)

# Layer 6
x = Conv2D(512, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_6', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_6')(x)
x = LeakyReLU(alpha=0.1)(x)
x = MaxPooling2D(padding='same', pool_size=(2, 2), strides=(1,1))(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(1024 ,3 ,count ,x ,weights_file , True)

# Layer 7
x = Conv2D(1024, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_7', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_7')(x)
x = LeakyReLU(alpha=0.1)(x)

route7 = x

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(256 ,1 ,count ,x ,weights_file , True)

# Layer 8
x = Conv2D(256, (1,1), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_8', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_8')(x)
x = LeakyReLU(alpha=0.1)(x)

convd8 = x

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(512 ,3 ,count ,x ,weights_file , True)

# Layer 9
x = Conv2D(512, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_9', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_9')(x)
x = LeakyReLU(alpha=0.1)(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(255 ,1 ,count ,x ,weights_file , False)

# Layer 10
x = Conv2D(255, (1,1), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same',name='conv_10', use_bias=True)(x)
n1, n2 = int(x.shape[1]), int(x.shape[2])
# x = Activation('linear')(x)

convd10 = x

# Layer 11 - yolo
n1, n2 = int(x.shape[1]), int(x.shape[2])
n3 = 3
classes = 80
n4 = (4 + 1 + classes)
yolo0 = Reshape((n1, n2, n3, n4))(x)

# Layer 12 - route
# https://github.com/AlexeyAB/darknet/issues/279#issuecomment-397248821
# Get input for next Conv layer from layer -4 index previous = Layer 7

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(128 ,1 ,count ,convd8 ,weights_file , True)

# Layer 13 - route layer
x = Conv2D(128, (1,1), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_13', use_bias=False)(convd8)
x = BatchNormalization(weights=bn_weight_list, name='norm_13')(x)
x = LeakyReLU(alpha=0.1)(x)

# Layer 14 - upsample
# x = ZeroPadding2D(((1, 0), (1, 0)))(convd10) # Adjust padding model for darknet.
x = UpSampling2D(size=(2, 2))(x)

# Layer 15 - route
# https://github.com/AlexeyAB/darknet/issues/279#issuecomment-397248821
# Get input for next Conv layer from layer -1 index previous = Layer 14 and Layer 8
# skip_connection = Lambda(space_to_depth_x2)(x)
# x = concatenate([route8, skip_connection])
x = concatenate([convd5, x])

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(256 ,3 ,count ,x ,weights_file , True)

# Layer 16
x = Conv2D(256, (3,3), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_16', use_bias=False)(x)
x = BatchNormalization(weights=bn_weight_list, name='norm_16')(x)
x = LeakyReLU(alpha=0.1)(x)

# ******** Apply weights function *************
conv_weights, bn_weight_list, count = setWeights(255 ,1 ,count ,x ,weights_file , False)

# Layer 17
x = Conv2D(255, (1,1), strides=(1,1), kernel_regularizer=l2(weight_decay), weights=conv_weights, padding='same', name='conv_17', use_bias=True)(x)

# Layer 18 - yolo
n1, n2 = int(x.shape[1]), int(x.shape[2])
yolo1 = Reshape((n1, n2, 3, 85))(x)

# Create and save model.
model = Model(inputs=[input_image],  outputs=[yolo0, yolo1])

# Export to hd5 file
model.save('{}'.format('yolotest.h5'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [ ]:
# !cp yolotest.h5 ../YOLOv3/data/yolo.h5
#%cd ../
#!rm -rf yolo3_tiny
#!rm -rf YOLOv3
#!ls -l

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 416, 416, 16) 432         input_1[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 416, 416, 16) 64          conv_1[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 416, 416, 16) 0           norm_1[0][0]                     
____________________________________________________________________________________________

In [ ]:
# Check to see if all weights have been read.
remaining_weights = len(weights_file.read()) / 4

print('Read {} of {} from Darknet weights.'.format(count, count +
remaining_weights))

weights_file.close()

Read 8858734 of 8858734.0 from Darknet weights.


##Perform detection on image

[Based on code from xiaochus github repo](https://github.com/xiaochus/YOLOv3)

###Load yolo model from file

In [ ]:
yolo = load_model('yolotest.h5')
# yolo = model

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


###Image processing functions
[Based on code from xiaochus github repo](https://github.com/xiaochus/YOLOv3)

In [ ]:
# obj_threshold
# t1 = 0.6
t1 = 0.1
# nms_threshold
t2 = 0.5
    
def sigmoid(x):
        """sigmoid.

        # Arguments
            x: Tensor.

        # Returns
            numpy ndarray.
        """
        return 1 / (1 + np.exp(-x))

def process_feats(out, anchors, mask):
    """process output features.

    # Arguments
        out: Tensor (N, N, 3, 4 + 1 +80), output feature map of yolo.
        anchors: List, anchors for box.
        mask: List, mask for anchors.

    # Returns
        boxes: ndarray (N, N, 3, 4), x,y,w,h for per box.
        box_confidence: ndarray (N, N, 3, 1), confidence for per box.
        box_class_probs: ndarray (N, N, 3, 80), class probs for per box.
    """
    grid_h, grid_w, num_boxes = map(int, out.shape[1: 4])

    anchors = [anchors[i] for i in mask]
    anchors_tensor = np.array(anchors).reshape(1, 1, len(anchors), 2)

    # Reshape to batch, height, width, num_anchors, box_params.
    out = out[0]
    box_xy = sigmoid(out[..., :2])
    box_wh = np.exp(out[..., 2:4])
    box_wh = box_wh * anchors_tensor

    box_confidence = sigmoid(out[..., 4])
    box_confidence = np.expand_dims(box_confidence, axis=-1)
    box_class_probs = sigmoid(out[..., 5:])

    col = np.tile(np.arange(0, grid_w), grid_w).reshape(-1, grid_w)
    row = np.tile(np.arange(0, grid_h).reshape(-1, 1), grid_h)

    col = col.reshape(grid_h, grid_w, 1, 1).repeat(3, axis=-2)
    row = row.reshape(grid_h, grid_w, 1, 1).repeat(3, axis=-2)
    grid = np.concatenate((col, row), axis=-1)

    box_xy += grid
    box_xy /= (grid_w, grid_h)
    box_wh /= (416, 416)
    box_xy -= (box_wh / 2.)
    boxes = np.concatenate((box_xy, box_wh), axis=-1)

    return boxes, box_confidence, box_class_probs
    
def nms_boxes(boxes, scores):
    """Suppress non-maximal boxes.

    # Arguments
        boxes: ndarray, boxes of objects.
        scores: ndarray, scores of objects.

    # Returns
        keep: ndarray, index of effective boxes.
    """
    x = boxes[:, 0]
    y = boxes[:, 1]
    w = boxes[:, 2]
    h = boxes[:, 3]

    areas = w * h
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)

        xx1 = np.maximum(x[i], x[order[1:]])
        yy1 = np.maximum(y[i], y[order[1:]])
        xx2 = np.minimum(x[i] + w[i], x[order[1:]] + w[order[1:]])
        yy2 = np.minimum(y[i] + h[i], y[order[1:]] + h[order[1:]])

        w1 = np.maximum(0.0, xx2 - xx1 + 1)
        h1 = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w1 * h1

        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= t2)[0]
        order = order[inds + 1]

    keep = np.array(keep)

    return keep
      
def filter_boxes(boxes, box_confidences, box_class_probs):
    """Filter boxes with object threshold.

    # Arguments
        boxes: ndarray, boxes of objects.
        box_confidences: ndarray, confidences of objects.
        box_class_probs: ndarray, class_probs of objects.

    # Returns
        boxes: ndarray, filtered boxes.
        classes: ndarray, classes for boxes.
        scores: ndarray, scores for boxes.
    """
    box_scores = box_confidences * box_class_probs
    box_classes = np.argmax(box_scores, axis=-1)
    box_class_scores = np.max(box_scores, axis=-1)
    pos = np.where(box_class_scores >= t1)

    boxes = boxes[pos]
    classes = box_classes[pos]
    scores = box_class_scores[pos]

    return boxes, classes, scores
      
def yolo_out(outs, shape):
    """Process output of yolo base net.

    # Argument:
        outs: output of yolo base net.
        shape: shape of original image.

    # Returns:
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
    """
    masks = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
    anchors = [[10, 13], [16, 30], [33, 23], [30, 61], [62, 45],
               [59, 119], [116, 90], [156, 198], [373, 326]]

    boxes, classes, scores = [], [], []

    for out, mask in zip(outs, masks):
        b, c, s = process_feats(out, anchors, mask)
        b, c, s = filter_boxes(b, c, s)
        boxes.append(b)
        classes.append(c)
        scores.append(s)

    boxes = np.concatenate(boxes)
    classes = np.concatenate(classes)
    scores = np.concatenate(scores)

    # Scale boxes back to original image shape.
    width, height = shape[0], shape[1]
    image_dims = [width, height, width, height]
    boxes = boxes * image_dims

    nboxes, nclasses, nscores = [], [], []
    for c in set(classes):
        inds = np.where(classes == c)
        b = boxes[inds]
        c = classes[inds]
        s = scores[inds]

        keep = nms_boxes(b, s)

        nboxes.append(b[keep])
        nclasses.append(c[keep])
        nscores.append(s[keep])

    if not nclasses and not nscores:
        return None, None, None

    boxes = np.concatenate(nboxes)
    classes = np.concatenate(nclasses)
    scores = np.concatenate(nscores)

    return boxes, classes, scores

def draw(image, boxes, scores, classes, all_classes):
    
  """Draw the boxes on the image.
    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
  """
#uncomment to use a custom font uploaded to this notebook directory
  fnt =ImageFont.truetype('./fonts/Ubuntu-Bold.ttf', 18)

  for box, score, cl in zip(boxes, scores, classes):
  
      x, y, w, h = box

      top = max(0, np.floor(x + 0.5).astype(int))
      left = max(0, np.floor(y + 0.5).astype(int))
      right = min(image.size[0], np.floor(x + w + 0.5).astype(int))
      bottom = min(image.size[1], np.floor(y + h + 0.5).astype(int))

      draw = ImageDraw.Draw(image)
      draw.rectangle([(top, left), (right, bottom)], outline=(0,255,0))#blue rectangle
      draw.text((top, left - 6), '{0} {1:.2f}'.format(all_classes[cl], score), font=fnt, fill=(0, 0, 255))
#      draw.text((top, left - 6), '{0} {1:.2f}'.format(all_classes[cl], score), fill=(0, 0, 255))
#         draw.rectangle([(left, top), (right, bottom)], outline=(0,0,255))#blue rectangle
      print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
      print('box coordinate x,y,w,h: {0}'.format(box))

  print()

  

###Load Image
Resize, reduce and expand image.

In [ ]:
#Pick one image to process by uncommenting it
#input_image_name = 'person.jpg'
#input_image_name = 'giraffe.jpg'
#input_image_name = 'dog.jpg'
#input_image_name = 'eagle.jpg'
#input_image_name = 'horses.jpg'
input_image_name = 'toysoldiers.jpg'

size = (416, 416)
image_src = Image.open("images/" + input_image_name)
orig_size = image_src.size
image_thumb = image_src.resize(size, Image.BICUBIC)
image_thumb.save("out/thumb_" + input_image_name, "JPEG") #save thumbnail version
image = np.array(image_thumb, dtype='float32')
image /= 255.
image = np.expand_dims(image, axis=0)


###Get prediction output

In [ ]:
# Raw Prediction Output
output = yolo.predict(image)


In [ ]:
with open('coco_classes.txt') as f:
    class_names = f.readlines()
all_classes = [c.strip() for c in class_names]

# Processed Output
image_thumb = Image.open("out/thumb_" + input_image_name) #open thumbnail image
thumb_size = image_thumb.size
boxes, classes, scores = yolo_out(output, thumb_size)  #process thumbnail image
#boxes, classes, scores = yolo_out(output, orig_size)
if boxes is not None:
  draw(image_thumb, boxes, scores, classes, all_classes) #annotate thumbnail image
  #draw(image_src, boxes, scores, classes, all_classes)

# Display processed image output
# image_src.show()  
#image_src.save("out/" + input_image_name, "JPEG")
image_thumb.save("out/" + input_image_name, "JPEG")

class: person, score: 0.59
box coordinate x,y,w,h: [168.31674671 140.2295084   77.26895118 123.73338103]
class: person, score: 0.41
box coordinate x,y,w,h: [351.90662384  35.0162816   69.95378542 116.17601037]

